In [1]:
# %pip install domolibrary --upgrade
# %pip install domolibrary_execution --upgrade

In [2]:
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj
import domolibrary.client.DomoAuth as dmda

config_auth = await dxdj.generate_domo_auth('domo-community', domojupyter_fn = dj)
config_auth

🎉 full_auth token retrieved from domo-community ⚙️


DomoFullAuth(domo_instance='domo-community', url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true', token_name='full_auth', user_id=None, is_valid_token=True, domo_username='jae@datacrew.space')

In [3]:
import domolibrary.classes.DomoJupyter as dmdj

workspace_id = '70afbd1d-e85c-4846-8a0e-a46987011676'

async def get_workspace_access_token_accounts(workspace_id, auth):
    domo_workspace = await dmdj.DomoJupyterWorkspace.get_by_id(workspace_id, auth = auth)
        
    return [dja for dja in domo_workspace.account_configuration if dja.domo_account and dja.domo_account.data_provider_type == 'domo-access-token']
   
    
workspace_access_token_accounts = await get_workspace_access_token_accounts(workspace_id=workspace_id, auth = config_auth)
test_dja = workspace_access_token_accounts[0]
workspace_access_token_accounts[0:5]

[DomoJupyterAccount(account_id=96, alias='sdk_domo-community', is_exists=True, domo_account=DomoAccount_Credential(id=96, name='sdk_domo-community', data_provider_type='domo-access-token', created_dt=datetime.datetime(2024, 2, 27, 0, 8, 54), modified_dt=datetime.datetime(2025, 2, 10, 21, 15, 47), owners=None, is_admin_summary=False, target_instance=None, is_valid_full_auth=None, is_valid_token_auth=None, target_auth=None, target_user=None))]

In [4]:
from typing import Callable
import domolibrary_execution.client.ExecutionError as ExecutionError

def extract_domo_instance_from_account_name(account_name):
    if not account_name.startswith('sdk_'):
        raise ExecutionError('account name does not start with sdk_')
    
    return account_name.split('_')[-1]
    

def populate_cred_config(dja : dmdj.DomoJupyterAccount,
                         domojupyter_fn : Callable
):
    dja.read_creds(domojupyter_fn = domojupyter_fn)
    
    dja.domo_account.target_instance = extract_domo_instance_from_account_name(dja.domo_account.name)
    
    return dja

populate_cred_config(dja = test_dja,
                     domojupyter_fn = dj)

DomoJupyterAccount(account_id=96, alias='sdk_domo-community', is_exists=True, domo_account=DomoAccount_Credential(id=96, name='sdk_domo-community', data_provider_type='domo-access-token', created_dt=datetime.datetime(2024, 2, 27, 0, 8, 54), modified_dt=datetime.datetime(2025, 2, 10, 21, 15, 47), owners=None, is_admin_summary=False, target_instance='domo-community', is_valid_full_auth=None, is_valid_token_auth=None, target_auth=None, target_user=None))

In [5]:
import domolibrary_extensions.utils.xkcd_password as xkcd

def generate_new_password():
    return xkcd.generate_domo_password(process_fn=xkcd.process_domo_password_fn, delimiter="-")

generate_new_password()

'LASSO-octane-swerve2025.'

In [6]:

async def fix_password(
    domo_account,
    backup_auth: dmda.DomoAuth = None,
    new_password: str = None,
    force_password_required: bool = False,
    is_update_account: bool = True,
    debug_api: bool = False,
    **kwargs,
):

    if domo_account.is_valid_full_auth:
        return True

    user_email = domo_account.Config.username

    if force_password_required and not user_email:
        raise dxde.ExecutionException(
            f"no username for {domo_account.id} - {domo_account.name}"
        )

    print(f"rotating {user_email} passsword in {domo_account.target_instance}")

    new_password = new_password or generate_new_password()

    await domo_account.update_target_user_password(
        new_password=new_password,
        is_test_auths=False,
        is_update_account=is_update_account,
        backup_auth=backup_auth,
        debug_api=debug_api,
    )

In [7]:
# domo_access_tokens = await dmat.DomoAccessTokens(auth = test_dja.domo_account.target_auth).get()
[dat.display_name for dat in domo_access_tokens]

NameError: name 'domo_access_tokens' is not defined

In [ ]:
import domolibrary.classes.DomoAccessToken as dmat
import asyncio

async def fix_token(
    domo_account,
    backup_auth: dmda.DomoAuth = None,
    
    token_name : str = None,
    force_token_required : bool = False,
    token_update_threshold: int = 7 ,

    is_update_account : bool = False,
    debug_api: bool = False,
    **kwargs,
):
    
    token_name = token_name or domo_account.name
    
    domo_access_tokens = await dmat.DomoAccessTokens(auth = domo_account.target_auth).get()
    
    print(token_name)
    print(len(domo_access_tokens))
          
    dat = next((dat for dat in domo_access_tokens if dat.name == token_name), None)
    
    # if not dat:
    #     print(token_name)
    #     print(domo_access_tokens)
    
    return dat

await test_dja.domo_account.test_auths()
await fix_token(domo_account = test_dja.domo_account)


In [ ]:


repair_functions = [fix_password]

async def test_and_fix_domo_account(
    dja: dmdj.DomoJupyterAccount,
    backup_auth: dmda.DomoAuth = None,
    force_password_required: bool = False,
    new_password: str = None,
    debug_api : bool = False
):
    domo_account = dja.domo_account

    await domo_account.test_auths()

    for fn in repair_functions:
        await fn(
            domo_account=domo_account,
            backup_auth=backup_auth,
            force_password_required=force_password_required,
            new_password=new_password,
            is_update_account = True,
            debug_api = debug_api
        )
        
    await asyncio.sleep(5)
    await domo_account.test_auths()


# await test_and_fix_domo_account(dja=test_dja, debug_api = True)
